<a href="https://colab.research.google.com/github/MdShakil-Hossain/366-Final-Lab/blob/main/CSE_366_Final_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install torch torchvision matplotlib grad-cam

In [16]:
#Mounting
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
#Path_to_dataset
dataset_path = '/content/drive/MyDrive/CSE 366 Lab/caltech-101'

In [22]:
import torch
from torch import nn
from torch.utils.data import random_split, DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

In [23]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [40]:
#Load the dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

#Number of classes
num_classes = len(dataset.classes)
print(f"Number of classes in the dataset: {num_classes}")

Number of classes in the dataset: 102


In [41]:
#Split dataset into training, validation and testing sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

In [42]:
#DataLoaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)

In [43]:
#Using ResNet50 as the pre-trained model
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(2048, num_classes)

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [39]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1, Loss: 3.5704
Epoch 2, Loss: 2.7344
Epoch 3, Loss: 2.3662
Epoch 4, Loss: 2.0330
Epoch 5, Loss: 1.7701
Epoch 6, Loss: 1.5812
Epoch 7, Loss: 1.4904
Epoch 8, Loss: 1.3383
Epoch 9, Loss: 1.2293
Epoch 10, Loss: 1.1270


In [44]:
model.eval()
val_accuracy = 0
with torch.no_grad():
    for images, labels in val_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        val_accuracy += (preds == labels).sum().item()
val_accuracy /= len(val_data)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

Validation Accuracy: 1.31%


In [45]:
test_accuracy = 0
y_pred, y_true = [], []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        test_accuracy += (preds == labels).sum().item()
        y_pred.extend(preds.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

test_accuracy /= len(test_data)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 0.55%


In [46]:
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

report = classification_report(y_true, y_pred, target_names=dataset.classes)
print("Classification Report:\n", report)

Confusion Matrix:
 [[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Classification Report:
                    precision    recall  f1-score   support

BACKGROUND_Google       1.00      0.02      0.05        41
            Faces       0.00      0.00      0.00        34
       Faces_easy       0.00      0.00      0.00        38
         Leopards       0.00      0.00      0.00        24
       Motorbikes       0.00      0.00      0.00        78
        accordion       0.00      0.00      0.00         7
        airplanes       0.00      0.00      0.00        83
           anchor       0.00      0.00      0.00         6
              ant       0.00      0.00      0.00         5
           barrel       0.00      0.00      0.00         6
             bass       0.00      0.00      0.00         8
           beaver       0.00      0.00      0.00         5
        binocular       0.00      0.00      0.00         5
          

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [47]:
#ResNet50 architecture
target_layer = model.layer4[-1]
cam = GradCAM(model=model, target_layer=target_layer)

#Display Grad-CAM for the first test image
for images, labels in test_loader:
    grayscale_cam = cam(input_tensor=images, target_category=labels[0])
    cam_image = show_cam_on_image(images[0].permute(1, 2, 0).cpu().numpy(), grayscale_cam)
    plt.imshow(cam_image)
    plt.show()
    break

Exception ignored in: <function BaseCAM.__del__ at 0x7d60026e9440>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pytorch_grad_cam/base_cam.py", line 189, in __del__
    self.activations_and_grads.release()
    ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'GradCAM' object has no attribute 'activations_and_grads'


TypeError: GradCAM.__init__() got an unexpected keyword argument 'target_layer'

In [48]:
torch.save(model.state_dict(), "caltech101_resnet50.pth")